# Digit Recognizer

In [40]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.optim import lr_scheduler
from sklearn.model_selection import train_test_split
from torchvision import transforms
from torch.autograd import Variable
import torch.nn.functional as F

import sys
import math

### Data Loader

In [41]:
class mnistData(Dataset):
    
    def __init__(self, data, transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.RandomRotation(degrees = (15,30)),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5,), std=(0.5,))
    ])):
        self.data = data
        self.transform = transform
        
        # distinguish Test_data and Train_data
        if len(data.columns) == test_columns:
            self.image = data.values.reshape((-1,28,28)).astype(np.uint8)[:,:,:,None]
            self.label = None
        else:
            self.image = data.iloc[:,1:].values.reshape((-1,28,28)).astype(np.uint8)[:,:,:,None]
            self.label = torch.from_numpy(data.iloc[:,0].values)
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        if self.label is not None:
            return self.transform(self.image[idx]), self.label[idx]
        else:
            return self.transform(self.image[idx])

In [42]:
train_data = pd.read_csv('/Users/Adamyae/Desktop/Demos/Digit Recognizer/train.csv')
test_data = pd.read_csv('/Users/Adamyae/Desktop/Demos/Digit Recognizer/test.csv')

test_columns = len(test_data.columns)

print("total image pixels: ", test_columns)

total image pixels:  784


In [43]:
batch_size = 64

train_dataset = mnistData(train_data,transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.RandomRotation(degrees = (15,30)),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5,), std=(0.5,))
    ]))
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)

test_dataset = mnistData(test_data)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=False)


In [44]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
          
        self.classifier = nn.Sequential(
            nn.Dropout(p = 0.5),
            nn.Linear(64 * 7 * 7, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(p = 0.5),
            nn.Linear(512, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(p = 0.5),
            nn.Linear(512, 10),
        )
          
        for m in self.features.children():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
        
        for m in self.classifier.children():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform(m.weight)
            elif isinstance(m, nn.BatchNorm1d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
                

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        
        return x     

In [45]:
model = CNN()

exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)


optimizer = optim.Adam(model.parameters(), lr=0.003)

criterion = nn.CrossEntropyLoss()

if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()

/Users/Adamyae/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:45: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.


In [51]:
def train(epoch):
    model.train()
    exp_lr_scheduler.step()

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        
        if torch.cuda.is_available():
            data = data.cuda()
            target = target.cuda()
        
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        
        loss.backward()
        optimizer.step()
        
        if (batch_idx + 1)% 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, (batch_idx + 1) * len(data), len(train_loader.dataset),
                100. * (batch_idx + 1) / len(train_loader), loss.item()))

In [52]:
def evaluate(data_loader):
    model.eval()
    loss = 0
    correct = 0
    
    for data, target in data_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        if torch.cuda.is_available():
            data = data.cuda()
            target = target.cuda()
        
        output = model(data)
        
        loss += F.cross_entropy(output, target, size_average=False).item()

        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
    loss /= len(data_loader.dataset)
        
    print('\nAverage loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)\n'.format(
        loss, correct, len(data_loader.dataset),
        100. * correct / len(data_loader.dataset)))

In [53]:
n_epochs = 50

for epoch in range(n_epochs):
    train(epoch)
    evaluate(train_loader)

Train Epoch: 0 [6400/42000 (15%)]	Loss: 0.104517
Train Epoch: 0 [12800/42000 (30%)]	Loss: 0.091916
Train Epoch: 0 [19200/42000 (46%)]	Loss: 0.085336
Train Epoch: 0 [25600/42000 (61%)]	Loss: 0.023714
Train Epoch: 0 [32000/42000 (76%)]	Loss: 0.199634
Train Epoch: 0 [38400/42000 (91%)]	Loss: 0.029944


/Users/Adamyae/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  import sys



Average loss: 0.0269, Accuracy: 41660/42000 (99.190%)

Train Epoch: 1 [6400/42000 (15%)]	Loss: 0.041224
Train Epoch: 1 [12800/42000 (30%)]	Loss: 0.012975
Train Epoch: 1 [19200/42000 (46%)]	Loss: 0.133953
Train Epoch: 1 [25600/42000 (61%)]	Loss: 0.016240
Train Epoch: 1 [32000/42000 (76%)]	Loss: 0.097422
Train Epoch: 1 [38400/42000 (91%)]	Loss: 0.049487

Average loss: 0.0273, Accuracy: 41647/42000 (99.160%)

Train Epoch: 2 [6400/42000 (15%)]	Loss: 0.154739
Train Epoch: 2 [12800/42000 (30%)]	Loss: 0.094597
Train Epoch: 2 [19200/42000 (46%)]	Loss: 0.153287
Train Epoch: 2 [25600/42000 (61%)]	Loss: 0.203798
Train Epoch: 2 [32000/42000 (76%)]	Loss: 0.010068
Train Epoch: 2 [38400/42000 (91%)]	Loss: 0.054983

Average loss: 0.0250, Accuracy: 41661/42000 (99.193%)

Train Epoch: 3 [6400/42000 (15%)]	Loss: 0.029204
Train Epoch: 3 [12800/42000 (30%)]	Loss: 0.011092
Train Epoch: 3 [19200/42000 (46%)]	Loss: 0.020761
Train Epoch: 3 [25600/42000 (61%)]	Loss: 0.124630
Train Epoch: 3 [32000/42000 (76%)]	

Train Epoch: 23 [38400/42000 (91%)]	Loss: 0.003898

Average loss: 0.0074, Accuracy: 41893/42000 (99.745%)

Train Epoch: 24 [6400/42000 (15%)]	Loss: 0.002291
Train Epoch: 24 [12800/42000 (30%)]	Loss: 0.024492
Train Epoch: 24 [19200/42000 (46%)]	Loss: 0.000894
Train Epoch: 24 [25600/42000 (61%)]	Loss: 0.001226
Train Epoch: 24 [32000/42000 (76%)]	Loss: 0.001427
Train Epoch: 24 [38400/42000 (91%)]	Loss: 0.003399

Average loss: 0.0053, Accuracy: 41928/42000 (99.829%)

Train Epoch: 25 [6400/42000 (15%)]	Loss: 0.005476
Train Epoch: 25 [12800/42000 (30%)]	Loss: 0.028954
Train Epoch: 25 [19200/42000 (46%)]	Loss: 0.000898
Train Epoch: 25 [25600/42000 (61%)]	Loss: 0.005149
Train Epoch: 25 [32000/42000 (76%)]	Loss: 0.047132
Train Epoch: 25 [38400/42000 (91%)]	Loss: 0.001599

Average loss: 0.0065, Accuracy: 41916/42000 (99.800%)

Train Epoch: 26 [6400/42000 (15%)]	Loss: 0.007014
Train Epoch: 26 [12800/42000 (30%)]	Loss: 0.005566
Train Epoch: 26 [19200/42000 (46%)]	Loss: 0.012143
Train Epoch: 26 [25

Train Epoch: 46 [25600/42000 (61%)]	Loss: 0.005715
Train Epoch: 46 [32000/42000 (76%)]	Loss: 0.055611
Train Epoch: 46 [38400/42000 (91%)]	Loss: 0.000304

Average loss: 0.0042, Accuracy: 41943/42000 (99.864%)

Train Epoch: 47 [6400/42000 (15%)]	Loss: 0.000220
Train Epoch: 47 [12800/42000 (30%)]	Loss: 0.001703
Train Epoch: 47 [19200/42000 (46%)]	Loss: 0.008906
Train Epoch: 47 [25600/42000 (61%)]	Loss: 0.023150
Train Epoch: 47 [32000/42000 (76%)]	Loss: 0.011560
Train Epoch: 47 [38400/42000 (91%)]	Loss: 0.001029

Average loss: 0.0033, Accuracy: 41960/42000 (99.905%)

Train Epoch: 48 [6400/42000 (15%)]	Loss: 0.005036
Train Epoch: 48 [12800/42000 (30%)]	Loss: 0.000819
Train Epoch: 48 [19200/42000 (46%)]	Loss: 0.004230
Train Epoch: 48 [25600/42000 (61%)]	Loss: 0.006106
Train Epoch: 48 [32000/42000 (76%)]	Loss: 0.003668
Train Epoch: 48 [38400/42000 (91%)]	Loss: 0.007553

Average loss: 0.0032, Accuracy: 41964/42000 (99.914%)

Train Epoch: 49 [6400/42000 (15%)]	Loss: 0.001782
Train Epoch: 49 [12

In [54]:
def prediciton(data_loader):
    model.eval()
    test_pred = torch.LongTensor()
    
    for i, data in enumerate(data_loader):
        data = Variable(data, volatile=True)
        if torch.cuda.is_available():
            data = data.cuda()
            
        output = model(data)
        
        pred = output.cpu().data.max(1, keepdim=True)[1]
        test_pred = torch.cat((test_pred, pred), dim=0)
        
    return test_pred

In [55]:
test_pred = prediciton(test_loader)

/Users/Adamyae/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  


In [56]:
out_df = pd.DataFrame(np.c_[np.arange(1, len(test_dataset)+1)[:,None], test_pred.numpy()], 
                      columns=['ImageId', 'Label'])

In [57]:
out_df.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,0
4,5,3


In [58]:
out_df.to_csv('submission.csv', index=False)